<a href="https://colab.research.google.com/github/JafarunnishaShaik/Fraud-Transaction-Detection/blob/main/Fraud_Transaction_Detection_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [9]:
#loading the dataset to a pandas DataFrame
credit_card_data = pd.read_pickle('/content/2018-04-01.pkl')

In [ ]:
#first 5 rows of the dataset
credit_card_data.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,0,2018-04-01 00:00:31,596,3156,57.16,31,0,0,0
1,1,2018-04-01 00:02:10,4961,3412,81.51,130,0,0,0
2,2,2018-04-01 00:07:56,2,1365,146.00,476,0,0,0
3,3,2018-04-01 00:09:29,4128,8737,64.49,569,0,0,0
4,4,2018-04-01 00:10:34,927,9906,50.99,634,0,0,0


In [10]:
credit_card_data.tail()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
9483,9483,2018-04-01 23:56:50,3289,6699,48.69,86210,0,0,0
9484,9484,2018-04-01 23:58:14,3230,6664,85.97,86294,0,0,0
9485,9485,2018-04-01 23:58:31,296,3702,120.88,86311,0,0,0
9486,9486,2018-04-01 23:59:28,2557,146,8.02,86368,0,0,0
9487,9487,2018-04-01 23:59:51,554,8248,19.20,86391,0,0,0


In [11]:
#data set informations
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9488 entries, 0 to 9487
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TRANSACTION_ID     9488 non-null   int64         
 1   TX_DATETIME        9488 non-null   datetime64[ns]
 2   CUSTOMER_ID        9488 non-null   object        
 3   TERMINAL_ID        9488 non-null   object        
 4   TX_AMOUNT          9488 non-null   float64       
 5   TX_TIME_SECONDS    9488 non-null   object        
 6   TX_TIME_DAYS       9488 non-null   object        
 7   TX_FRAUD           9488 non-null   int64         
 8   TX_FRAUD_SCENARIO  9488 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 741.2+ KB


In [12]:
#checking the number of missing values in each column
credit_card_data.isnull().sum()

,0
TRANSACTION_ID,0
TX_DATETIME,0
CUSTOMER_ID,0
TERMINAL_ID,0
TX_AMOUNT,0
TX_TIME_SECONDS,0
TX_TIME_DAYS,0
TX_FRAUD,0
TX_FRAUD_SCENARIO,0


In [13]:
#distribution of legit transactions & fradulent transactions
credit_card_data['TX_FRAUD'].value_counts()

,count
TX_FRAUD,
0,9485
1,3


This data set highly unblanced

0----->Normal Transaction
1----->Fraudulent transaction

In [14]:
#Separating the data for analysis
legit = credit_card_data[credit_card_data.TX_FRAUD == 0]
fraud = credit_card_data[credit_card_data.TX_FRAUD == 1]

In [ ]:
print(legit.shape)
print(fraud.shape)

(9485, 9)
(3, 9)


In [16]:
#satatical measures of the data
legit.TX_AMOUNT.describe()

,TX_AMOUNT
count,9485.000000
mean,53.195756
std,39.392437
min,0.130000
25%,21.190000
50%,45.580000
75%,76.920000
max,218.120000


In [17]:
fraud.TX_AMOUNT.describe()

,TX_AMOUNT
count,3.000000
mean,224.690000
std,2.161874
min,222.260000
25%,223.835000
50%,225.410000
75%,225.905000
max,226.400000


In [ ]:
from google.colab import drive
drive.mount('/content/drive')